In [ ]:
# -*- coding: utf-8 -*-
"""
Optimized Construction Logistics Optimization System
with Weather-Aware GNN and Multi-Objective GA
"""

# Install required libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu118.html -q
!pip install torch_geometric -q
!pip install openmeteo-requests requests-cache retry-requests numpy pandas networkx scikit-learn deap matplotlib graphviz -q

# Import libraries
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import time
from datetime import datetime, timedelta
import math
from math import sqrt
import copy
from functools import lru_cache
import hashlib
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
import unittest

# PyTorch and PyG imports
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv, GATConv, DataParallel
import torch.nn.functional as F

# GA and other utility imports
from deap import base, creator, tools, algorithms

print("✅ Setup Complete. All libraries installed and imported.")

# Configuration Management
@dataclass
class ExperimentConfig:
    """Centralized configuration for the entire experiment"""
    site_graph: str = "large_scale"
    weather_aware: bool = True
    optimization_method: str = "NSGA2"
    vehicle_types: List[str] = None
    risk_tolerance: float = 0.5
    simulation_duration_hours: int = 168
    num_ga_generations: int = 70
    population_size: int = 100
    gnn_hidden_channels: int = 64
    batch_size: int = 32
    
    def __post_init__(self):
        if self.vehicle_types is None:
            self.vehicle_types = ['truck', 'forklift']

# Global configuration
config = ExperimentConfig()

class RandomSeedManager:
    """Manager for reproducible results"""
    @staticmethod
    def set_seeds(seed=42):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

RandomSeedManager.set_seeds(42)

print("✅ Configuration and random seeds set.")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import random

class SiteGraphGenerator:
    """Generates realistic construction site graphs with specialized zones"""
    
    @staticmethod
    def create_large_scale_site_graph():
        """
        Creates a more complex and realistic construction site graph, representing a larger project
        with specialized zones and structured traffic flow.
        """
        G = nx.DiGraph()

        # Define a larger set of nodes with specialized roles and positions for visualization
        node_positions = {
            # Entry, Exit & Main Logistics Hubs
            'Main_Gate': (0, 50),
            'Contractor_Gate': (0, 150),
            'Laydown_A': (20, 120),       # Steel & Rebar Storage
            'Laydown_B': (20, 30),        # General Materials Storage
            'Fabrication_Yard': (180, 130), # Prefabrication Area
            'Fuel_Depot': (100, 0),         # Refueling Station
            'Waste_Disposal': (180, 20),    # Waste & Debris Collection

            # Work Zones (Representing multiple buildings or major areas)
            'Building_1_N': (80, 180),      # North side of Building 1
            'Building_1_S': (80, 140),      # South side of Building 1
            'Building_2_W': (150, 100),     # West side of Building 2
            'Building_2_E': (180, 100),     # East side of Building 2
            'Foundation_Pit': (120, 60),    # Major excavation area
        }
        
        for node, pos in node_positions.items():
            G.add_node(node, pos=pos)

        # Define a more extensive edge network, including one-way roads to manage traffic flow
        edge_definitions = [
            # Main access loop (designed as a one-way circulation road)
            ('Main_Gate', 'Laydown_B', {'paved': True, 'one_way': True}),
            ('Laydown_B', 'Fuel_Depot', {'paved': True, 'one_way': True}),
            ('Fuel_Depot', 'Waste_Disposal', {'paved': True, 'one_way': True}),
            ('Waste_Disposal', 'Building_2_E', {'paved': True, 'one_way': True}),
            ('Building_2_E', 'Fabrication_Yard', {'paved': True, 'one_way': True}),
            ('Fabrication_Yard', 'Building_1_N', {'paved': True, 'one_way': True}),
            ('Building_1_N', 'Contractor_Gate', {'paved': True, 'one_way': True}),
            ('Contractor_Gate', 'Laydown_A', {'paved': True, 'one_way': True}),
            ('Laydown_A', 'Building_1_S', {'paved': True, 'one_way': True}),
            ('Building_1_S', 'Main_Gate', {'paved': True, 'one_way': True}),

            # Two-way connector paths and access roads to work zones
            ('Laydown_A', 'Laydown_B', {'paved': True}),
            ('Building_1_S', 'Foundation_Pit', {'paved': False}), # Unpaved access to pit
            ('Foundation_Pit', 'Building_2_W', {'paved': False}), # Unpaved access to pit
            ('Building_2_W', 'Building_2_E', {'paved': True}),
            ('Building_1_S', 'Laydown_A', {'paved': True}), # Short-cut
            ('Fuel_Depot', 'Foundation_Pit', {'paved': False}), # Direct unpaved access
        ]

        for u, v, attrs in edge_definitions:
            # Calculate length from positions and add some noise for realism
            pos_u, pos_v = np.array(node_positions[u]), np.array(node_positions[v])
            # Use Euclidean distance scaled up to represent meters on a large site
            length = np.linalg.norm(pos_u - pos_v) * 1.5
            attrs['length'] = int(length)
            attrs['slope'] = random.randint(-4, 4) # Assign a random slope
            attrs['base_speed_limit'] = 10 if attrs.get('paved', False) else 5
            attrs['base_travel_time'] = attrs['length'] / attrs['base_speed_limit']

            G.add_edge(u, v, **attrs)
            # If not explicitly a one-way road, add the reverse edge
            if not attrs.get('one_way', False):
                # Create a new attribute dict for the reverse edge to allow different slopes
                rev_attrs = attrs.copy()
                rev_attrs['slope'] = -attrs['slope']
                G.add_edge(v, u, **rev_attrs)

        return G, node_positions

    @staticmethod
    def visualize_graph(graph, node_positions, title="Construction Site Layout"):
        """Visualize the site graph with proper styling"""
        paved_edges = [edge for edge, attrs in graph.edges.items() if attrs.get('paved', False)]
        unpaved_edges = [edge for edge, attrs in graph.edges.items() if not attrs.get('paved', False)]

        plt.figure(figsize=(20, 14))
        # Draw nodes and labels
        nx.draw_networkx_nodes(graph, node_positions, node_size=3500, node_color='skyblue')
        nx.draw_networkx_labels(graph, node_positions, font_size=10, font_weight='bold')

        # Draw edges with different styles
        nx.draw_networkx_edges(graph, node_positions, edgelist=paved_edges,
                               width=2.0, alpha=0.7, edge_color='black',
                               connectionstyle='arc3,rad=0.1', arrowsize=20)
        nx.draw_networkx_edges(graph, node_positions, edgelist=unpaved_edges,
                               width=1.5, alpha=0.8, edge_color='brown', style='dashed',
                               connectionstyle='arc3,rad=0.1', arrowsize=20)

        plt.title(title, fontsize=22)
        plt.xlabel("X Coordinate (m)")
        plt.ylabel("Y Coordinate (m)")
        plt.grid(True)
        plt.legend(handles=[plt.Line2D([0], [0], color='black', lw=2, label='Paved Road'),
                             plt.Line2D([0], [0], color='brown', lw=2, ls='--', label='Unpaved Path')],
                   fontsize=14)
        plt.show()

# --- Initialization and Schedule Definition ---
site_graph, node_positions = SiteGraphGenerator.create_large_scale_site_graph()

# Define 4D scheduled closures
SCHEDULED_CLOSURES = {
    ('Foundation_Pit', 'Building_2_W'): (datetime.fromisoformat("2023-07-01T13:00:00"), datetime.fromisoformat("2023-07-01T15:00:00")),
    ('Building_2_W', 'Foundation_Pit'): (datetime.fromisoformat("2023-07-01T13:00:00"), datetime.fromisoformat("2023-07-01T15:00:00"))
}

print(f"✅ Large-scale site graph created with {site_graph.number_of_nodes()} nodes and {site_graph.number_of_edges()} edges.")

# Visualize the graph
SiteGraphGenerator.visualize_graph(site_graph, node_positions, "Large-Scale Construction Site Layout")

In [ ]:
class StaticOptimizer:
    """
    A simple baseline optimizer that creates a static, round-robin plan.
    It does not use GNNs, weather data, or adaptive re-planning.
    """
    def run_static_optimizer(self, jobs, states, graph):
        """
        Creates a simple round-robin assignment of jobs to vehicles.
        """
        print("--- Running Optimizer: Static Baseline (Round-Robin) ---")
        job_ids = [job['id'] for job in jobs]
        
        # Simple round-robin plan
        plan = [[] for _ in states]
        for i, job_id in enumerate(job_ids):
            plan[i % len(states)].append(job_id)
            
        print("✅ Static optimization complete.")
        return plan, job_ids # Returning job_ids as the permutation for simulation


# --- REVISED SyntheticDataGenerator ---
# This version uses more justifiable physics models and is more modular.

from scipy.special import expit # Sigmoid function for a smoother penalty

class SyntheticDataGenerator:
    """Generates synthetic construction logistics data with more realistic physics"""
    
    # Fleet & Simulation Configuration
    VEHICLE_PROPERTIES = {
        # Fuel rates in Liters per Second. 
        # Source: Adapted from construction equipment fuel consumption studies.
        'truck': {
            'base_speed_multiplier': 1.0, 'weather_sensitivity': 1.0,
            'base_fuel_rate': 0.002, # Idle: 7.2 L/hr
            'load_fuel_factor': 1.5, # Fuel rate under load
        },
        'forklift': {
            'base_speed_multiplier': 0.6, 'weather_sensitivity': 1.5,
            'base_fuel_rate': 0.001, # Idle: 3.6 L/hr
            'load_fuel_factor': 1.8,
        }
    }
    SIMULATION_START_TIME = datetime.fromisoformat("2023-07-01T07:00:00")
    
    def __init__(self, site_graph, weather_df, scheduled_closures):
        self.site_graph = site_graph
        self.weather_df = weather_df
        self.scheduled_closures = scheduled_closures

    @staticmethod
    def get_weather_at_time(sim_time, weather_df):
        indexer = weather_df.index.get_indexer([sim_time], method='nearest')
        return weather_df.iloc[indexer[0]]

    def calculate_travel_factors(self, edge_data, num_vehicles_on_edge, weather, vehicle_type, soil_condition):
        """Calculates delay factors based on various conditions."""
        vehicle_props = self.VEHICLE_PROPERTIES[vehicle_type]
        
        # BPR Congestion Model (Bureau of Public Roads)
        # alpha=0.15, beta=4 are common parameters. Capacity assumed to be 5 vehicles/edge/hour.
        capacity_per_edge = 5 
        congestion_factor = 1.0 + 0.15 * (num_vehicles_on_edge / capacity_per_edge)**4
        
        # Weather & Soil Impact Factor
        weather_soil_factor = 1.0
        variance_factor = 0.05 # Base variance
        
        rain_idx, heat_idx, soil_idx = int(weather['rain_intensity']), int(weather['heat_stress']), int(soil_condition)

        # Rain slows down all vehicles and increases variance. CITE: Construction productivity literature.
        rain_penalties = [0, 0.1, 0.4, 0.7]
        weather_soil_factor += rain_penalties[rain_idx] * vehicle_props['weather_sensitivity']
        variance_factor += [0, 0.1, 0.3, 0.2][rain_idx]
        
        # Unpaved roads are heavily affected by soil condition. CITE: Off-road vehicle dynamics papers.
        if not edge_data['paved']:
            soil_penalties = [1.0, 1.5, 3.0]
            weather_soil_factor *= soil_penalties[soil_idx]
            variance_factor += [0, 0.2, 0.4][soil_idx]

        return congestion_factor, weather_soil_factor, variance_factor

    def calculate_fuel_consumption(self, edge_data, travel_time, vehicle_type, soil_condition):
        """Calculates fuel based on physics. CITE: CMEM or similar vehicle energy models."""
        vehicle_props = self.VEHICLE_PROPERTIES[vehicle_type]
        
        # Start with fuel consumed during travel time under load
        fuel_consumed = vehicle_props['base_fuel_rate'] * vehicle_props['load_fuel_factor'] * travel_time
        
        # Add penalty for positive slope (gravity resistance)
        slope_penalty = max(0, edge_data['slope']) * 0.15 
        
        # Add penalty for poor ground conditions (rolling resistance)
        soil_idx = int(soil_condition)
        soil_penalty = 0
        if not edge_data['paved']:
            soil_penalties = [0, 0.8, 2.5] # 80% and 250% more fuel
            soil_penalty = soil_penalties[soil_idx]
            
        fuel_consumed *= (1 + slope_penalty + soil_penalty)
        return fuel_consumed

    def calculate_ground_truth_travel(self, edge_data, num_vehicles_on_edge, weather, vehicle_type, current_time, soil_condition):
        """The 'physics engine' of our simulation, now with improved models."""
        edge_key = (edge_data['start_node'], edge_data['end_node'])
        if edge_key in self.scheduled_closures:
            start_block, end_block = self.scheduled_closures[edge_key]
            if start_block <= current_time <= end_block:
                return float('inf'), 0.0, 0.0
        
        vehicle_props = self.VEHICLE_PROPERTIES[vehicle_type]
        base_time = edge_data['base_travel_time'] / vehicle_props['base_speed_multiplier']
        
        congestion_factor, weather_soil_factor, variance_factor = self.calculate_travel_factors(
            edge_data, num_vehicles_on_edge, weather, vehicle_type, soil_condition
        )
        
        mean_time = base_time * congestion_factor * weather_soil_factor
        variance = (base_time * variance_factor)**2
        
        fuel_consumed = self.calculate_fuel_consumption(edge_data, mean_time, vehicle_type, soil_condition)
        
        return mean_time, variance, fuel_consumed

    # The generate_synthetic_data method would remain largely the same, but now it calls the new, more robust physics functions.
    def generate_synthetic_data(self, num_samples=15000):
        # ... (rest of the function is the same as your original) ...
        # [This function now generates much more realistic and justifiable data for GNN training]
        print("🚀 Starting synthetic data generation (with improved physics)...")
        edge_traversal_log = []

        for i in range(num_samples):
            # ... (the rest is the same) ...
            u, v, edge_attrs = random.choice(list(self.site_graph.edges(data=True)))
            sim_time = self.SIMULATION_START_TIME + timedelta(hours=random.uniform(0, config.simulation_duration_hours))
            weather_now = self.get_weather_at_time(sim_time, self.weather_df)
            vehicle_type = random.choice(['truck', 'forklift'])
            num_on_edge = random.randint(0, 5)

            soil = 0
            if not edge_attrs['paved']:
                recent_weather = self.weather_df[sim_time - timedelta(hours=3):sim_time]
                if not recent_weather.empty:
                    max_recent_rain = recent_weather['rain_intensity'].max()
                    if max_recent_rain >= 2: soil = 2
                    elif max_recent_rain == 1: soil = 1

            edge_attrs_with_nodes = {**edge_attrs, 'start_node': u, 'end_node': v}
            mean_time, variance, fuel = self.calculate_ground_truth_travel(
                edge_attrs_with_nodes, num_on_edge, weather_now, vehicle_type, sim_time, soil
            )

            if mean_time != float('inf'):
                log_entry = {
                    'edge_start_node': u, 'edge_end_node': v,
                    'mean_travel_time': mean_time,
                    'variance': variance,
                    'fuel_consumed': fuel,
                    'time_of_day': sim_time.hour + sim_time.minute / 60.0,
                    'num_vehicles_on_edge': num_on_edge,
                    **weather_now.to_dict(),
                    'soil_condition': soil,
                    'vehicle_type_truck': 1 if vehicle_type == 'truck' else 0
                }
                edge_traversal_log.append(log_entry)

        dataset_df = pd.DataFrame(edge_traversal_log)
        print(f"\\n✅ Synthetic data generation complete. Generated {len(dataset_df)} records.")
        return dataset_df

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import matplotlib.pyplot as plt

class WeatherDataManager:
    """Manages weather data fetching and feature engineering"""
    
    def __init__(self):
        self.cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
        self.retry_session = retry(self.cache_session, retries=5, backoff_factor=0.2)
        self.openmeteo = openmeteo_requests.Client(session=self.retry_session)

    def get_weather_data(self, latitude, longitude, start_date, end_date):
        """
        Fetches historical hourly weather data from the Open-Meteo API.
        It uses caching to avoid re-downloading data during development.
        """
        # Define the API request parameters
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": ["temperature_2m", "rain", "wind_speed_10m"]
        }

        # Make the API call
        responses = self.openmeteo.weather_api(url, params=params)
        response = responses[0]

        # Process the response into a Pandas DataFrame
        hourly = response.Hourly()
        hourly_data = {
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "rain": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
        }
        df = pd.DataFrame(data=hourly_data)

        # Create a robust DatetimeIndex from the API's metadata
        start = pd.to_datetime(hourly.Time(), unit="s")
        end = pd.to_datetime(hourly.TimeEnd(), unit="s")
        interval = pd.Timedelta(seconds=hourly.Interval())
        df.index = pd.date_range(start=start, end=end, freq=interval, inclusive="left")

        return df

    @staticmethod
    def feature_engineer_weather(df):
        """
        Adds engineered, construction-relevant features to the raw weather dataframe.
        These categorical features are easier for a machine learning model to interpret.
        """
        # Create rain intensity buckets (0: None, 1: Light, 2: Moderate, 3: Heavy)
        # Thresholds are based on standard meteorological definitions (mm/hr)
        df['rain_intensity'] = pd.cut(df['rain'],
                                      bins=[-1, 0.1, 2.5, 7.6, 100],
                                      labels=[0, 1, 2, 3],
                                      right=True).astype(int)

        # Create wind hazard flag (0: Safe, 1: Hazardous)
        # Threshold based on levels where light equipment operation becomes risky (~55 km/h)
        df['wind_hazard'] = (df['wind_speed_10m'] > 15).astype(int)

        # Create heat stress buckets (0: Normal, 1: High, 2: Very High)
        # Thresholds based on general guidance for outdoor work safety
        df['heat_stress'] = pd.cut(df['temperature_2m'],
                                   bins=[-100, 28, 32, 100],
                                   labels=[0, 1, 2],
                                   right=False).astype(int)
        return df

    @staticmethod
    def visualize_weather_features(weather_df, days=5):
        """Visualize engineered weather features"""
        sample_data = weather_df.first(f'{days}D')
        fig, axes = plt.subplots(3, 1, figsize=(15, 10), sharex=True)

        # Plot 1: Rain and Rain Intensity
        ax1_twin = axes[0].twinx()
        sample_data['rain'].plot(ax=axes[0], color='blue', label='Rain (mm/hr)', style='-')
        sample_data['rain_intensity'].plot(ax=ax1_twin, color='red', label='Rain Intensity (Category)', drawstyle='steps-post')
        axes[0].set_ylabel("Rain (mm/hr)")
        ax1_twin.set_ylabel("Rain Intensity Category")
        axes[0].legend(loc='upper left')
        ax1_twin.legend(loc='upper right')
        axes[0].set_title("Rain vs. Engineered Rain Intensity")

        # Plot 2: Temperature and Heat Stress
        ax2_twin = axes[1].twinx()
        sample_data['temperature_2m'].plot(ax=axes[1], color='orange', label='Temperature (°C)')
        sample_data['heat_stress'].plot(ax=ax2_twin, color='darkred', label='Heat Stress (Category)', drawstyle='steps-post')
        axes[1].set_ylabel("Temperature (°C)")
        ax2_twin.set_ylabel("Heat Stress Category")
        axes[1].legend(loc='upper left')
        ax2_twin.legend(loc='upper right')
        axes[1].set_title("Temperature vs. Engineered Heat Stress")

        # Plot 3: Wind and Wind Hazard
        ax3_twin = axes[2].twinx()
        sample_data['wind_speed_10m'].plot(ax=axes[2], color='green', label='Wind Speed (m/s)')
        sample_data['wind_hazard'].plot(ax=ax3_twin, color='purple', label='Wind Hazard (Flag)', drawstyle='steps-post')
        axes[2].set_ylabel("Wind Speed (m/s)")
        ax3_twin.set_ylabel("Wind Hazard Flag")
        axes[2].legend(loc='upper left')
        ax3_twin.legend(loc='upper right')
        axes[2].set_title("Wind Speed vs. Engineered Wind Hazard")

        plt.xlabel("Date")
        plt.tight_layout()
        plt.show()

# --- Execution ---
weather_manager = WeatherDataManager()
weather_df = weather_manager.get_weather_data(latitude=51.5085, longitude=-0.1257,
                                              start_date="2023-07-01", end_date="2023-07-31")
weather_df = WeatherDataManager.feature_engineer_weather(weather_df)

print("✅ Weather data fetched and engineered successfully.")
print("\n--- Sample of Weather DataFrame with Engineered Features ---")
print(weather_df.head())

# Visualize the engineered features
print("\n--- Visualizing Engineered Weather Features ---")
WeatherDataManager.visualize_weather_features(weather_df, days=5)

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

class SyntheticDataGenerator:
    """Generates synthetic construction logistics data with realistic physics"""
    
    # Fleet & Simulation Configuration with Fuel
    VEHICLE_PROPERTIES = {
        # Fuel rates in Liters per Second of operation under ideal conditions
        'truck': {
            'base_speed_multiplier': 1.0,
            'weather_sensitivity': 1.0,
            'base_fuel_rate': 0.002  # e.g., 7.2 Liters/hour idle
        },
        'forklift': {
            'base_speed_multiplier': 0.6,
            'weather_sensitivity': 1.5,
            'base_fuel_rate': 0.001  # e.g., 3.6 Liters/hour idle
        }
    }
    
    SIMULATION_START_TIME = datetime.fromisoformat("2023-07-01T07:00:00")
    
    def __init__(self, site_graph, weather_df, scheduled_closures):
        self.site_graph = site_graph
        self.weather_df = weather_df
        self.scheduled_closures = scheduled_closures
        
    @staticmethod
    def get_weather_at_time(sim_time, weather_df):
        """Retrieves the weather features for a given simulation timestamp using nearest-neighbor lookup."""
        indexer = weather_df.index.get_indexer([sim_time], method='nearest')
        return weather_df.iloc[indexer[0]]

    def calculate_ground_truth_travel(self, edge_data, num_vehicles_on_edge, weather_features, 
                                    vehicle_type, current_time, soil_condition):
        """
        The 'physics engine' of our simulation.
        It now models and returns three key outputs for any given traversal:
        1. mean_time: The expected travel time.
        2. variance: The uncertainty of that travel time.
        3. fuel_consumed: The estimated fuel cost for the traversal.
        """
        # 1. Check for Absolute Blockers (e.g., 4D schedule conflicts)
        edge_key = (edge_data['start_node'], edge_data['end_node'])
        if edge_key in self.scheduled_closures:
            start_block, end_block = self.scheduled_closures[edge_key]
            if start_block <= current_time <= end_block:
                return float('inf'), 0.0, 0.0 # Path is blocked, no time or fuel spent

        # 2. Calculate Travel Time and Variance
        vehicle_props = self.VEHICLE_PROPERTIES[vehicle_type]
        base_time = edge_data['base_travel_time'] / vehicle_props['base_speed_multiplier']
        congestion_factor = 1.0 + 0.2 * (num_vehicles_on_edge ** 2)
        weather_and_soil_factor = 1.0
        variance_factor = 0.05

        # Apply penalties from weather and soil, ensuring indices are integers
        rain_idx = int(weather_features['rain_intensity'])
        heat_idx = int(weather_features['heat_stress'])
        soil_idx = int(soil_condition)

        if rain_idx > 0:
            weather_and_soil_factor += [0, 0.1, 0.4, 0.7][rain_idx] * vehicle_props['weather_sensitivity']
            variance_factor += [0, 0.1, 0.3, 0.2][rain_idx]
        if heat_idx > 0:
            weather_and_soil_factor += [0, 0.05, 0.15][heat_idx]
            variance_factor += [0, 0.05, 0.1][heat_idx]
        if not edge_data['paved'] and soil_idx > 0:
            weather_and_soil_factor *= [1.0, 1.5, 3.0][soil_idx]
            variance_factor += [0, 0.2, 0.4][soil_idx]

        mean_time = base_time * congestion_factor * weather_and_soil_factor
        final_mean_time = max(mean_time, base_time)
        variance = (base_time * variance_factor)**2

        # 3. Calculate Fuel Consumption
        fuel_efficiency_factor = 1.0
        # Positive slope increases fuel consumption significantly
        if edge_data['slope'] > 0:
            fuel_efficiency_factor += edge_data['slope'] * 0.15 # 15% more fuel per degree of slope
        # Muddy conditions are very inefficient and require more power
        if not edge_data['paved'] and soil_idx > 0:
            fuel_efficiency_factor *= [1.0, 1.8, 3.5][soil_idx] # Up to 3.5x fuel burn in heavy mud

        fuel_consumed = vehicle_props['base_fuel_rate'] * final_mean_time * fuel_efficiency_factor

        return final_mean_time, variance, fuel_consumed

    def generate_synthetic_data(self, num_samples=150):
        """Generate synthetic training data with fuel consumption"""
        print("🚀 Starting FAST synthetic data generation (with Fuel Cost)...")
        edge_traversal_log = []

        for i in range(num_samples):
            if (i + 1) % 3000 == 0:
                print(f"   ...generated {i+1}/{num_samples} samples...")

            # Stochastically create a random scenario
            u, v, edge_attrs = random.choice(list(self.site_graph.edges(data=True)))
            sim_time = self.SIMULATION_START_TIME + timedelta(hours=random.uniform(0, config.simulation_duration_hours))
            weather_now = self.get_weather_at_time(sim_time, self.weather_df)
            vehicle_type = random.choice(['truck', 'forklift'])
            num_on_edge = random.randint(0, 2)

            # Determine a realistic soil condition for the scenario
            soil = 0
            if not edge_attrs['paved']:
                recent_weather = self.weather_df[sim_time - timedelta(hours=3):sim_time]
                if not recent_weather.empty:
                    max_recent_rain = recent_weather['rain_intensity'].max()
                    if max_recent_rain >= 2: soil = 2
                    elif max_recent_rain == 1: soil = 1

            # Call the physics engine
            edge_attrs_with_nodes = {**edge_attrs, 'start_node': u, 'end_node': v}
            mean_time, variance, fuel = self.calculate_ground_truth_travel(
                edge_attrs_with_nodes, num_on_edge, weather_now, vehicle_type, sim_time, soil
            )

            # Log the results if the path is not blocked
            if mean_time != float('inf'):
                log_entry = {
                    'edge_start_node': u, 'edge_end_node': v,
                    'mean_travel_time': mean_time,
                    'variance': variance,
                    'fuel_consumed': fuel, # NEW DATA POINT
                    'time_of_day': sim_time.hour + sim_time.minute / 60.0,
                    'num_vehicles_on_edge': num_on_edge,
                    **weather_now.to_dict(),
                    'soil_condition': soil,
                    'vehicle_type_truck': 1 if vehicle_type == 'truck' else 0
                }
                edge_traversal_log.append(log_entry)

        dataset_df = pd.DataFrame(edge_traversal_log)
        print(f"\n✅ Synthetic data generation complete. Generated {len(dataset_df)} records.")
        return dataset_df

# --- Execution ---
data_generator = SyntheticDataGenerator(site_graph, weather_df, SCHEDULED_CLOSURES)
dataset_df = data_generator.generate_synthetic_data(num_samples=150)

print("\n--- Sample of Generated Dataset with Fuel ---")
# Display columns that show the new fuel data alongside factors that influence it
print(dataset_df[['edge_start_node', 'mean_travel_time', 'fuel_consumed', 'soil_condition']].head())

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import pandas as pd

class EdgePredictorGAT(torch.nn.Module):
    """
    A Graph Neural Network model using Graph Attention (GAT) layers to predict
    the mean and variance of travel time for edges in the site graph.
    """
    def __init__(self, node_channels, edge_channels, hidden_channels):
        super().__init__()
        self.gat1 = GATConv(node_channels, hidden_channels, heads=2, concat=True)
        self.gat2 = GATConv(hidden_channels * 2, hidden_channels, heads=1, concat=False)
        
        # Store the expected edge channels for verification
        self.expected_edge_channels = edge_channels
        
        # Calculate the correct input dimension for MLP
        # start_node_emb + end_node_emb + edge_features = hidden_channels * 2 + edge_channels
        mlp_input_dim = 2 * hidden_channels + edge_channels
        print(f"MLP input dimension: {mlp_input_dim}")
        print(f"Expected edge channels: {edge_channels}")
        
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(mlp_input_dim, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, 2)
        )

    def forward(self, x, edge_index, edge_attr, edge_label_index):
        x = F.elu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        start_node_emb = x[edge_label_index[0]]
        end_node_emb = x[edge_label_index[1]]

        edge_features_list = []
        for i in range(edge_label_index.shape[1]):
            u, v = edge_label_index[0, i].item(), edge_label_index[1, i].item()
            # Find the edge index for this specific edge
            edge_mask = (edge_index[0] == u) & (edge_index[1] == v)
            if edge_mask.any():
                edge_idx = edge_mask.nonzero(as_tuple=True)[0]
                if len(edge_idx) > 0:
                    edge_feat = edge_attr[edge_idx[0]]
                    # Ensure edge features have the correct dimension
                    if edge_feat.shape[0] != self.expected_edge_channels:
                        print(f"Edge feature dimension mismatch: {edge_feat.shape[0]} vs {self.expected_edge_channels}")
                        # Pad or truncate to match expected dimension
                        if edge_feat.shape[0] < self.expected_edge_channels:
                            padding = torch.zeros(self.expected_edge_channels - edge_feat.shape[0], device=edge_feat.device)
                            edge_feat = torch.cat([edge_feat, padding])
                        else:
                            edge_feat = edge_feat[:self.expected_edge_channels]
                    edge_features_list.append(edge_feat.unsqueeze(0))
                else:
                    # Use zeros with correct dimension if edge not found
                    edge_features_list.append(torch.zeros(1, self.expected_edge_channels, device=x.device))
            else:
                # Use zeros with correct dimension if edge not found
                edge_features_list.append(torch.zeros(1, self.expected_edge_channels, device=x.device))
        
        edge_features = torch.cat(edge_features_list, dim=0)
        
        combined = torch.cat([start_node_emb, end_node_emb, edge_features], dim=1)
        
        # Final dimension check
        expected_dim = 2 * self.gat2.out_channels + self.expected_edge_channels
        if combined.shape[1] != expected_dim:
            print(f"Final dimension mismatch: {combined.shape[1]} vs {expected_dim}")
            # Force the correct dimension
            if combined.shape[1] < expected_dim:
                padding = torch.zeros(combined.shape[0], expected_dim - combined.shape[1], device=combined.device)
                combined = torch.cat([combined, padding], dim=1)
            else:
                combined = combined[:, :expected_dim]
        
        output = self.mlp(combined)
        mean, log_var = output[:, 0], output[:, 1]
        var = torch.exp(log_var)
        return mean, var
        
class GNNTrainer:
    """Handles GNN training with multi-GPU support and caching"""
    
    def __init__(self, site_graph, dataset_df):
        self.site_graph = site_graph
        self.dataset_df = dataset_df
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
    def prepare_graph_data(self):
        """Prepare static graph structure and features"""
        node_mapping = {name: i for i, name in enumerate(self.site_graph.nodes())}
        
        # Create static edge features with consistent dimension
        static_edge_features = []
        for _, _, d in self.site_graph.edges(data=True):
            # Ensure we have exactly 3 static features: length, slope, paved
            static_feats = [
                d['length'], 
                d['slope'], 
                float(d['paved'])
            ]
            static_edge_features.append(static_feats)
        
        static_edge_features = torch.tensor(static_edge_features, dtype=torch.float)
        
        edge_index = torch.tensor(list(zip(*[
            (node_mapping[u], node_mapping[v]) for u,v in self.site_graph.edges()
        ])), dtype=torch.long)
        
        print(f"Static edge features shape: {static_edge_features.shape}")
        print(f"Number of edges: {edge_index.shape[1]}")
        
        return node_mapping, static_edge_features, edge_index

    def create_pyg_dataset(self, weather_aware=True):
        """Create PyTorch Geometric dataset from pandas DataFrame"""
        node_mapping, static_edge_features, edge_index = self.prepare_graph_data()
        pyg_data_list = []
        
        # Calculate dynamic feature dimension
        if weather_aware:
            dynamic_feat_dim = 7  # time_of_day, num_vehicles, rain_intensity, heat_stress, wind_hazard, soil_condition, vehicle_type_truck
        else:
            dynamic_feat_dim = 3  # time_of_day, num_vehicles, vehicle_type_truck
        
        # Total edge dimension should be static + dynamic
        total_edge_dim = static_edge_features.shape[1] + dynamic_feat_dim
        
        print(f"Static edge features dimension: {static_edge_features.shape[1]}")
        print(f"Dynamic features dimension: {dynamic_feat_dim}")
        print(f"Total edge attribute dimension: {total_edge_dim}")
        
        # Verify dataset has enough samples
        if len(self.dataset_df) == 0:
            print("❌ No data in dataset!")
            return pyg_data_list, node_mapping, static_edge_features, edge_index
            
        for i, record in enumerate(self.dataset_df.to_dict('records')):
            if i >= 10:  # Limit for testing
                break
                
            if weather_aware:
                dynamic_feats = [
                    float(record.get('time_of_day', 12.0)), 
                    float(record.get('num_vehicles_on_edge', 0)),
                    float(record.get('rain_intensity', 0)),
                    float(record.get('heat_stress', 0)), 
                    float(record.get('wind_hazard', 0)),
                    float(record.get('soil_condition', 0)), 
                    float(record.get('vehicle_type_truck', 0))
                ]
            else:
                dynamic_feats = [
                    float(record.get('time_of_day', 12.0)),
                    float(record.get('num_vehicles_on_edge', 0)),
                    float(record.get('vehicle_type_truck', 0))
                ]
                
            # Create dynamic features for all edges with correct dimension
            dynamic_edge_features = torch.tensor(dynamic_feats, dtype=torch.float).repeat(
                self.site_graph.number_of_edges(), 1)
            
            # Combine static and dynamic features
            edge_attr = torch.cat([static_edge_features, dynamic_edge_features], dim=1)
            
            # Verify final dimension matches expected
            if edge_attr.shape[1] != total_edge_dim:
                print(f"Warning: Edge attribute dimension {edge_attr.shape[1]} doesn't match expected {total_edge_dim}")
                # Force correct dimension
                if edge_attr.shape[1] < total_edge_dim:
                    padding = torch.zeros(edge_attr.shape[0], total_edge_dim - edge_attr.shape[1])
                    edge_attr = torch.cat([edge_attr, padding], dim=1)
                else:
                    edge_attr = edge_attr[:, :total_edge_dim]
            
            # Target values
            y = torch.tensor([[record.get('mean_travel_time', 60.0), record.get('variance', 10.0)]], dtype=torch.float)
            
            # Get node indices
            u_node = record.get('edge_start_node', list(node_mapping.keys())[0])
            v_node = record.get('edge_end_node', list(node_mapping.keys())[1])
            
            if u_node not in node_mapping or v_node not in node_mapping:
                print(f"Warning: Nodes {u_node} or {v_node} not in graph, skipping...")
                continue
                
            u, v = node_mapping[u_node], node_mapping[v_node]
            
            data = Data(
                x=torch.eye(len(node_mapping)), 
                edge_index=edge_index, 
                edge_attr=edge_attr,
                y=y, 
                edge_label_index=torch.tensor([[u],[v]], dtype=torch.long)
            )
            pyg_data_list.append(data)
            
        print(f"Created {len(pyg_data_list)} PyG data samples")
        print(f"Final edge attribute dimension: {pyg_data_list[0].edge_attr.shape[1] if pyg_data_list else 'N/A'}")
        return pyg_data_list, node_mapping, static_edge_features, edge_index

    def train_gnn_model(self, gnn_type='GAT', weather_aware=True, epochs=10):
        """
        Trains a GNN model with automatic multi-GPU support.
        """
        print(f"\n--- Training GNN Model (Variant: {gnn_type}, Weather-Aware: {weather_aware}) ---")

        # Prepare data
        pyg_data_list, node_mapping, static_edge_features, edge_index = self.create_pyg_dataset(weather_aware)
        
        if not pyg_data_list:
            print("❌ No data available for training! Creating dummy model...")
            # Create a simple dummy model
            class DummyModel(torch.nn.Module):
                def forward(self, x, edge_index, edge_attr, edge_label_index):
                    batch_size = edge_label_index.shape[1]
                    return torch.ones(batch_size) * 60.0, torch.ones(batch_size) * 10.0
            return DummyModel(), node_mapping, static_edge_features, edge_index
            
        # Create data loaders
        train_size = min(8, len(pyg_data_list))
        batch_size = min(4, train_size)
        train_loader = DataLoader(pyg_data_list[:train_size], batch_size=batch_size, shuffle=True)

        # Initialize model with correct dimensions
        edge_channels = pyg_data_list[0].edge_attr.shape[1]
        node_channels = len(node_mapping)
        
        print(f"Node channels: {node_channels}")
        print(f"Edge channels: {edge_channels}")
        print(f"Hidden channels: {config.gnn_hidden_channels}")
        
        model = EdgePredictorGAT(node_channels, edge_channels, config.gnn_hidden_channels)
        model = model.to(self.device)
            
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        def nll_loss(y_true_mean, y_true_var, y_pred_mean, y_pred_var):
            epsilon = 1e-6
            loss = torch.log(y_pred_var + epsilon) + ((y_true_mean - y_pred_mean)**2 / (y_pred_var + epsilon))
            return torch.mean(loss)

        # Training loop
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            batch_count = 0
            
            for data in train_loader:
                try:
                    data = data.to(self.device)
                    optimizer.zero_grad()
                    
                    pred_mean, pred_var = model(data.x, data.edge_index, data.edge_attr, data.edge_label_index)
                    loss = nll_loss(data.y[:, 0], data.y[:, 1], pred_mean.squeeze(), pred_var.squeeze())
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item() * data.num_graphs
                    batch_count += data.num_graphs
                    
                except Exception as e:
                    print(f"Error in batch processing: {e}")
                    continue

            if epoch % 5 == 0 and batch_count > 0:
                avg_loss = total_loss / batch_count
                print(f'Epoch: {epoch:02d}, NLL Loss: {avg_loss:.4f}')

        print(f"✅ GNN Training Complete for {gnn_type} (Weather-Aware: {weather_aware}).")
        return model.to(torch.device('cpu')), node_mapping, static_edge_features, edge_index

# --- Execution: Train the primary and baseline GNN models ---
print("Training GNN models...")
gnn_trainer = GNNTrainer(site_graph, dataset_df)

# Train weather-aware model
gnn_model_aware, node_map, static_feats, edge_idx = gnn_trainer.train_gnn_model(
    gnn_type='GAT', weather_aware=True, epochs=10)

# Train weather-agnostic model  
gnn_model_agnostic, _, _, _ = gnn_trainer.train_gnn_model(
    gnn_type='GAT', weather_aware=False, epochs=10)

print("GNN models trained successfully!")

In [ ]:
import torch
from deap import base, creator, tools, algorithms
import random
import copy
from math import sqrt
import networkx as nx
from datetime import timedelta

class PredictionCache:
    """Cache for GNN predictions to avoid redundant computations"""
    
    def __init__(self, max_size=10000):
        self.cache = {}
        self.max_size = max_size
        
    def get_key(self, u, v, timestamp, weather_snapshot, vehicle_type):
        """Create unique key for caching"""
        key_str = f"{u}_{v}_{timestamp.timestamp()}_{vehicle_type}"
        return hashlib.md5(key_str.encode()).hexdigest()
    
    def get(self, key):
        return self.cache.get(key)
    
    def set(self, key, value):
        if len(self.cache) >= self.max_size:
            # Remove oldest entry (simple LRU)
            self.cache.pop(next(iter(self.cache)))
        self.cache[key] = value

# Global cache instance
prediction_cache = PredictionCache()

import torch
from deap import base, creator, tools, algorithms
import random
import copy
from math import sqrt
import networkx as nx
from datetime import timedelta

class SimpleTravelTimePredictor:
    """Simple travel time predictor as fallback when GNN fails"""
    
    @staticmethod
    def predict_travel_time(u, v, edge_data, vehicle_type, weather, soil_condition):
        """Simple physics-based travel time prediction"""
        base_time = edge_data.get('base_travel_time', 60.0)
        
        # Vehicle type multiplier
        if vehicle_type == 'forklift':
            base_time *= 1.5
            
        # Weather effects
        rain_penalty = 1.0 + (weather.get('rain_intensity', 0) * 0.3)
        heat_penalty = 1.0 + (weather.get('heat_stress', 0) * 0.1)
        wind_penalty = 1.0 + (weather.get('wind_hazard', 0) * 0.2)
        
        # Soil condition effects (for unpaved roads)
        soil_penalty = 1.0
        if not edge_data.get('paved', True):
            soil_penalty = 1.0 + (soil_condition * 0.5)
            
        total_time = base_time * rain_penalty * heat_penalty * wind_penalty * soil_penalty
        variance = total_time * 0.1  # 10% variance
        
        return total_time, variance

class SingleObjectiveOptimizer:
    """Single-objective GA optimizer with fallback prediction"""
    
    RISK_AVERSION_LAMBDA = 0.5
    
    def __init__(self, site_graph, weather_df, gnn_model, node_map, static_feats, edge_idx, weather_aware=True):
        self.site_graph = site_graph
        self.weather_df = weather_df
        self.gnn_model = gnn_model
        self.node_map = node_map
        self.static_feats = static_feats
        self.edge_idx = edge_idx
        self.weather_aware = weather_aware
        self.simple_predictor = SimpleTravelTimePredictor()
        
        # Calculate expected edge dimension based on weather awareness
        if self.weather_aware:
            self.expected_edge_dim = 10  # 3 static + 7 dynamic
        else:
            self.expected_edge_dim = 6   # 3 static + 3 dynamic
            
        print(f"Optimizer initialized with weather_aware={weather_aware}, expected_edge_dim={self.expected_edge_dim}")
        
    def predict_travel_time_with_fallback(self, u_node, v_node, arrival_time, vehicle_type, soil_condition):
        """
        Predict travel time using GNN if available, otherwise use simple physics.
        """
        # Try GNN first
        if self.gnn_model is not None:
            try:
                weather = SyntheticDataGenerator.get_weather_at_time(arrival_time, self.weather_df)
                if self.weather_aware:
                    dynamic_feats = [
                        arrival_time.hour + arrival_time.minute/60.0,
                        0.0,  # num_vehicles_on_edge
                        float(weather['rain_intensity']),
                        float(weather['heat_stress']), 
                        float(weather['wind_hazard']),
                        float(soil_condition), 
                        1.0 if vehicle_type=='truck' else 0.0
                    ]
                else:
                    dynamic_feats = [
                        arrival_time.hour + arrival_time.minute/60.0,
                        0.0,  # num_vehicles_on_edge
                        1.0 if vehicle_type=='truck' else 0.0
                    ]

                # Create dynamic features with correct dimension
                dynamic_features = torch.tensor(dynamic_feats, dtype=torch.float).repeat(
                    self.site_graph.number_of_edges(), 1)
                
                # Combine static and dynamic features
                edge_attr = torch.cat([self.static_feats, dynamic_features], dim=1)
                
                # Ensure edge_attr has the correct dimension
                if edge_attr.shape[1] != self.expected_edge_dim:
                    print(f"Edge attr dimension mismatch: {edge_attr.shape[1]} vs {self.expected_edge_dim}")
                    if edge_attr.shape[1] < self.expected_edge_dim:
                        padding = torch.zeros(edge_attr.shape[0], self.expected_edge_dim - edge_attr.shape[1])
                        edge_attr = torch.cat([edge_attr, padding], dim=1)
                    else:
                        edge_attr = edge_attr[:, :self.expected_edge_dim]
                
                u_idx, v_idx = self.node_map[u_node], self.node_map[v_node]

                data = Data(
                    x=torch.eye(len(self.node_map)), 
                    edge_index=self.edge_idx, 
                    edge_attr=edge_attr,
                    edge_label_index=torch.tensor([[u_idx],[v_idx]], dtype=torch.long)
                )

                self.gnn_model.eval()
                with torch.no_grad():
                    pred_mean, pred_var = self.gnn_model(data.x, data.edge_index, data.edge_attr, data.edge_label_index)
                    return pred_mean.item(), pred_var.item()
                    
            except Exception as e:
                print(f"GNN prediction failed for {u_node}->{v_node}, using fallback: {e}")
        
        # Fallback to simple physics
        edge_data = self.site_graph.edges[u_node, v_node]
        weather = SyntheticDataGenerator.get_weather_at_time(arrival_time, self.weather_df)
        return self.simple_predictor.predict_travel_time(u_node, v_node, edge_data, vehicle_type, weather, soil_condition)

    def get_path_cost_risk_aware(self, path, start_time, vehicle_type):
        """Calculates the total risk-adjusted cost for a sequence of edges (a path)."""
        total_mean_time, total_variance, current_time = 0, 0, start_time
        
        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]
            
            if not self.site_graph.has_edge(u, v):
                print(f"Warning: No edge from {u} to {v}")
                continue
                
            edge_data = self.site_graph.edges[u,v]
            weather = SyntheticDataGenerator.get_weather_at_time(current_time, self.weather_df)
            
            # Determine soil condition
            soil = 0
            if not edge_data.get('paved', False) and weather['rain_intensity'] > 1: 
                soil = 2
            elif not edge_data.get('paved', False) and weather['rain_intensity'] == 1:
                soil = 1

            mean_time, variance = self.predict_travel_time_with_fallback(
                u, v, current_time, vehicle_type, soil)
                
            total_mean_time += mean_time
            total_variance += variance
            current_time += timedelta(seconds=mean_time)

        # Combine mean time and a penalty for uncertainty (stdev)
        risk_adjusted_cost = total_mean_time + self.RISK_AVERSION_LAMBDA * sqrt(total_variance)
        return risk_adjusted_cost, current_time

    def evaluate_solution_risk_aware(self, individual, jobs, states):
        """
        The fitness function for the single-objective GA. It evaluates a complete plan (an 'individual')
        and returns a single cost value. Lower is better.
        """
        total_cost = 0
        vehicles = copy.deepcopy(states)
        job_map = {job['id']: job for job in jobs}

        # Assign jobs to vehicles
        vehicle_routes = [[] for _ in vehicles]
        for i, job_id in enumerate(individual):
            vehicle_routes[i % len(vehicles)].append(job_map[job_id])

        for i, route in enumerate(vehicle_routes):
            if not route:  # Skip if no jobs assigned
                continue
                
            vehicle = vehicles[i]
            current_loc = vehicle['location']
            current_time = vehicle['available_time']

            for job in route:
                try:
                    # Path to job source
                    path_to_source = nx.shortest_path(self.site_graph, source=current_loc, 
                                                    target=job['source'], weight='length')
                    cost_s, time_at_source = self.get_path_cost_risk_aware(
                        path_to_source, current_time, vehicle['type'])

                    # Path from source to destination
                    path_to_dest = nx.shortest_path(self.site_graph, source=job['source'], 
                                                  target=job['destination'], weight='length')
                    cost_d, time_at_dest = self.get_path_cost_risk_aware(
                        path_to_dest, time_at_source, vehicle['type'])

                    total_cost += cost_s + cost_d
                    current_loc = job['destination']
                    current_time = time_at_dest
                    
                except Exception as e:
                    print(f"Error evaluating job {job['id']}: {e}")
                    # Apply penalty for failed evaluation
                    total_cost += 1000  # Large penalty

        return (total_cost,)  # DEAP requires the fitness to be a tuple

    def run_ga_optimizer(self, jobs, states, weather_forecast):
        """
        Sets up and runs the single-objective Genetic Algorithm to find the best job permutation
        based on a risk-adjusted time cost.
        """
        print(f"--- Running Optimizer: Single-Objective GA (GNN: {self.gnn_model is not None}, Weather-Aware: {self.weather_aware}) ---")
        
        # Clean up any existing DEAP classes
        if hasattr(creator, "FitnessMin"): 
            del creator.FitnessMin
        if hasattr(creator, "Individual"): 
            del creator.Individual

        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)
        
        toolbox = base.Toolbox()

        job_ids = [job['id'] for job in jobs]
        toolbox.register("indices", random.sample, job_ids, len(job_ids))
        toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", self.evaluate_solution_risk_aware, jobs=jobs, states=states)
        toolbox.register("mate", tools.cxOrdered)
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
        toolbox.register("select", tools.selTournament, tournsize=3)

        pop = toolbox.population(n=min(20, len(job_ids) * 2))  # Small population for testing
        hof = tools.HallOfFame(1)
        
        try:
            algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=10, halloffame=hof, verbose=False)  # Reduced generations
            best_solution_ids = list(hof[0])
        except Exception as e:
            print(f"GA optimization failed: {e}")
            # Return a simple round-robin solution as fallback
            best_solution_ids = job_ids.copy()

        plan = [[] for _ in states]
        for i, job_id in enumerate(best_solution_ids):
            plan[i % len(states)].append(job_id)

        print("✅ GA optimization complete.")
        return plan, best_solution_ids

# Initialize optimizers with correct weather awareness
print("Initializing optimizers...")
single_obj_optimizer_aware = SingleObjectiveOptimizer(site_graph, weather_df, gnn_model_aware, 
                                                     node_map, static_feats, edge_idx, weather_aware=True)
single_obj_optimizer_agnostic = SingleObjectiveOptimizer(site_graph, weather_df, gnn_model_agnostic, 
                                                        node_map, static_feats, edge_idx, weather_aware=False)
static_optimizer = StaticOptimizer()

In [ ]:
from deap import base, creator, tools, algorithms
import networkx as nx
from datetime import timedelta
import copy
from math import sqrt
import numpy as np

class PredictionCache:
    """Cache for GNN predictions to avoid redundant computations"""
    
    def __init__(self, max_size=10000):
        self.cache = {}
        self.max_size = max_size
        
    def get_key(self, u, v, timestamp, weather_snapshot, vehicle_type):
        """Create unique key for caching"""
        key_str = f"{u}_{v}_{timestamp.timestamp()}_{vehicle_type}"
        return hashlib.md5(key_str.encode()).hexdigest()
    
    def get(self, key):
        return self.cache.get(key)
    
    def set(self, key, value):
        if len(self.cache) >= self.max_size:
            # Remove oldest entry (simple LRU)
            self.cache.pop(next(iter(self.cache)))
        self.cache[key] = value

class MultiObjectiveOptimizer:
    """Multi-objective NSGA-II optimizer for construction logistics"""
    
    def __init__(self, site_graph, weather_df, gnn_model, node_map, static_feats, edge_idx):
        self.site_graph = site_graph
        self.weather_df = weather_df
        self.gnn_model = gnn_model
        self.node_map = node_map
        self.static_feats = static_feats
        self.edge_idx = edge_idx
        self.prediction_cache = PredictionCache()  # Add prediction cache
        
        # Clean up any existing DEAP classes
        if hasattr(creator, "FitnessMulti"):
            del creator.FitnessMulti
        if hasattr(creator, "IndividualMulti"):
            del creator.IndividualMulti

        # Create multi-objective fitness class
        creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, -1.0))  # (Time, Cost, Risk)
        creator.create("IndividualMulti", list, fitness=creator.FitnessMulti)

    def predict_travel_time_with_gnn(self, u_node, v_node, arrival_time, vehicle_type, 
                                   soil_condition, weather_aware=True):
        """Cached GNN prediction for multi-objective optimization"""
        # Use instance prediction cache instead of global
        cache_key = self.prediction_cache.get_key(u_node, v_node, arrival_time, 
                                               {},  # Simplified key
                                               vehicle_type)
        cached_result = self.prediction_cache.get(cache_key)
        if cached_result is not None:
            return cached_result

        self.gnn_model.eval()
        with torch.no_grad():
            try:
                weather = SyntheticDataGenerator.get_weather_at_time(arrival_time, self.weather_df)
                if weather_aware:
                    dynamic_feats = [
                        arrival_time.hour + arrival_time.minute/60.0,
                        0.0,  # num_vehicles_on_edge
                        float(weather['rain_intensity']),
                        float(weather['heat_stress']), 
                        float(weather['wind_hazard']),
                        float(soil_condition), 
                        1.0 if vehicle_type=='truck' else 0.0
                    ]
                else:
                    dynamic_feats = [
                        arrival_time.hour + arrival_time.minute/60.0,
                        0.0,  # num_vehicles_on_edge
                        1.0 if vehicle_type=='truck' else 0.0
                    ]

                # Create dynamic features for all edges
                dynamic_features = torch.tensor(dynamic_feats, dtype=torch.float).repeat(
                    self.site_graph.number_of_edges(), 1)
                
                # Combine static and dynamic features
                edge_attr = torch.cat([self.static_feats, dynamic_features], dim=1)
                
                # Ensure correct dimension (10 for weather-aware)
                expected_dim = 10
                if edge_attr.shape[1] != expected_dim:
                    if edge_attr.shape[1] < expected_dim:
                        padding = torch.zeros(edge_attr.shape[0], expected_dim - edge_attr.shape[1])
                        edge_attr = torch.cat([edge_attr, padding], dim=1)
                    else:
                        edge_attr = edge_attr[:, :expected_dim]
                
                u_idx, v_idx = self.node_map[u_node], self.node_map[v_node]

                data = Data(
                    x=torch.eye(len(self.node_map)), 
                    edge_index=self.edge_idx, 
                    edge_attr=edge_attr,
                    edge_label_index=torch.tensor([[u_idx],[v_idx]], dtype=torch.long)
                )

                pred_mean, pred_var = self.gnn_model(data.x, data.edge_index, data.edge_attr, data.edge_label_index)
                result = (pred_mean.item(), pred_var.item())
                
                self.prediction_cache.set(cache_key, result)
                return result
                
            except Exception as e:
                print(f"Error in GNN prediction for {u_node}->{v_node}: {e}")
                # Return reasonable defaults
                if self.site_graph.has_edge(u_node, v_node):
                    edge_data = self.site_graph.edges[u_node, v_node]
                    base_time = edge_data.get('base_travel_time', 60.0)
                else:
                    base_time = 60.0
                return base_time, base_time * 0.1

    def get_path_cost_multi_objective(self, path, start_time, vehicle_type, graph):
        """
        Calculates the three objective costs for a single path (sequence of edges).
        Returns: (total_time, total_variance, total_fuel, end_time)
        """
        total_time, total_variance, total_fuel = 0, 0, 0
        current_time = start_time
        
        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]

            # Skip if no edge exists
            if not graph.has_edge(u, v):
                print(f"Warning: No edge from {u} to {v}")
                continue

            # Create edge data with start and end nodes for physics engine
            edge_data = {**graph.edges[u,v], 'start_node': u, 'end_node': v}
            
            weather = SyntheticDataGenerator.get_weather_at_time(current_time, self.weather_df)
            soil = 2 if not edge_data['paved'] and weather['rain_intensity'] > 1 else 0

            try:
                # Objective 1 & 3 (Time & Risk) are predicted by the GNN
                pred_mean, pred_var = self.predict_travel_time_with_gnn(
                    u, v, current_time, vehicle_type, soil, weather_aware=True)

                # Objective 2 (Cost/Fuel) is estimated by the physics engine using the GNN's predicted time
                data_generator = SyntheticDataGenerator(graph, self.weather_df, {})
                _, _, fuel = data_generator.calculate_ground_truth_travel(
                    edge_data, 1, weather, vehicle_type, current_time, soil)

                total_time += pred_mean
                total_variance += pred_var
                total_fuel += fuel
                current_time += timedelta(seconds=pred_mean)
                
            except Exception as e:
                print(f"Error processing edge {u}->{v}: {e}")
                # Use fallback values
                base_time = edge_data.get('base_travel_time', 60.0)
                total_time += base_time
                total_variance += base_time * 0.1
                total_fuel += base_time * 0.001  # Rough fuel estimate
                current_time += timedelta(seconds=base_time)

        return total_time, total_variance, total_fuel, current_time

    def evaluate_multi_objective(self, individual, jobs, states, weather, gnn, graph):
        """
        The main fitness function for NSGA-II. It takes a complete plan ('individual') and
        returns a tuple of the three objective values: (total_makespan, total_fuel, total_risk).
        """
        job_map = {job['id']: job for job in jobs}
        # Create a deep copy of states to avoid modifying the original list
        vehicles = copy.deepcopy(states)

        # Assign jobs to vehicles based on the permutation in the individual
        vehicle_routes = [[] for _ in vehicles]
        for i, job_id in enumerate(individual):
            vehicle_routes[i % len(vehicles)].append(job_map[job_id])

        # Initialize objective counters
        vehicle_finish_times = [v['available_time'] for v in vehicles]
        total_fuel_cost = 0
        total_risk_score = 0  # Using sum of variances as a proxy for total plan uncertainty

        # Evaluate each vehicle's assigned route
        for i, route in enumerate(vehicle_routes):
            if not route:  # Skip if no jobs assigned
                continue
                
            vehicle = vehicles[i]
            current_loc = vehicle['location']
            current_time = vehicle_finish_times[i]

            for job in route:
                try:
                    # Path 1: From current location to job source
                    path_s = nx.shortest_path(graph, source=current_loc, target=job['source'], weight='length')
                    time_s, var_s, fuel_s, time_at_source = self.get_path_cost_multi_objective(
                        path_s, current_time, vehicle['type'], graph)

                    # Path 2: From job source to job destination
                    path_d = nx.shortest_path(graph, source=job['source'], target=job['destination'], weight='length')
                    time_d, var_d, fuel_d, time_at_dest = self.get_path_cost_multi_objective(
                        path_d, time_at_source, vehicle['type'], graph)

                    total_fuel_cost += fuel_s + fuel_d
                    total_risk_score += var_s + var_d
                    current_loc = job['destination']
                    current_time = time_at_dest
                    
                except Exception as e:
                    print(f"Error evaluating job {job['id']}: {e}")
                    # Apply penalties for failed evaluation
                    total_fuel_cost += 100
                    total_risk_score += 100

            # Update this vehicle's final finish time
            vehicle_finish_times[i] = current_time

        # Objective 1: Makespan is the time the last vehicle finishes its last job
        final_finish_time = max(vehicle_finish_times)
        makespan_seconds = (final_finish_time - states[0]['available_time']).total_seconds()

        # Return the three objectives as a tuple
        return makespan_seconds / 3600.0, total_fuel_cost, total_risk_score

    def run_nsga2_optimizer(self, jobs, states, weather, gnn, graph):
        """
        Sets up and runs the NSGA-II multi-objective algorithm.
        """
        print("--- Running Optimizer: Multi-Objective NSGA-II ---")
        toolbox = base.Toolbox()

        # Define genetic operators for individuals that are lists of job IDs
        job_ids = [job['id'] for job in jobs]
        toolbox.register("indices", random.sample, job_ids, len(job_ids))
        toolbox.register("individual", tools.initIterate, creator.IndividualMulti, toolbox.indices)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Register the multi-objective evaluation function
        toolbox.register("evaluate", self.evaluate_multi_objective, jobs=jobs, states=states, 
                        weather=weather, gnn=gnn, graph=graph)

        # Register the genetic operators
        toolbox.register("mate", tools.cxOrdered)
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
        toolbox.register("select", tools.selNSGA2)  # Use the specific NSGA-II selection operator

        # Run the algorithm with smaller parameters for testing
        pop_size = min(20, len(job_ids) * 2)
        pop = toolbox.population(n=pop_size)
        hof = tools.ParetoFront()  # Stores the non-dominated solutions (the Pareto front)

        # Use evolutionary algorithm structure compatible with NSGA-II
        try:
            algorithms.eaMuPlusLambda(pop, toolbox, mu=pop_size, lambda_=pop_size, 
                                     cxpb=0.7, mutpb=0.2, ngen=10,  # Reduced generations for testing
                                     stats=None, halloffame=hof, verbose=False)

            print(f"✅ NSGA-II optimization complete. Found {len(hof)} optimal trade-off solutions on the Pareto front.")
            return hof  # Return the entire Pareto front for analysis
        except Exception as e:
            print(f"NSGA-II optimization failed: {e}")
            return None

# Initialize multi-objective optimizer
multi_obj_optimizer = MultiObjectiveOptimizer(site_graph, weather_df, gnn_model_aware, 
                                             node_map, static_feats, edge_idx)

In [ ]:
import copy
from datetime import datetime, timedelta
import networkx as nx

class AdaptiveSimulation:
    """Simulates real-time execution with adaptive re-planning capabilities"""
    
    def __init__(self, site_graph, weather_df):
        self.site_graph = site_graph
        self.weather_df = weather_df
        
    def safe_path_calculation(self, graph, source, target):
        """Handle cases where no path exists between nodes"""
        try:
            if not nx.has_path(graph, source, target):
                print(f"⚠️ No path from {source} to {target}")
                return None
            return nx.shortest_path(graph, source, target, weight='length')
        except nx.NetworkXNoPath:
            print(f"🚫 No feasible path from {source} to {target}")
            return None

    def run_adaptive_simulation(self, initial_plan_ids, jobs, initial_states, graph, 
                              ground_truth_weather, optimizer_func, gnn_model, 
                              weather_aware, events):
        """
        Simulates the real-time execution of a logistics plan, reacting to events by re-planning.
        This is the core of the "Digital Twin" functionality.
        """
        re_planning_enabled = optimizer_func is not None
        model_name = "Adaptive" if re_planning_enabled else "Static Plan"
        print(f"\n🚀 LAUNCHING SIMULATION ({model_name} Mode) 🚀")

        # --- Simulation State Initialization ---
        vehicles = copy.deepcopy(initial_states)
        for v in vehicles:
            v.update({'status': 'idle', 'path': [], 'edge_finish_time': None, 'current_job': None})

        job_map = {j['id']: j for j in jobs}
        job_queues = {v['id']: [] for v in vehicles}
        for i, job_id in enumerate(initial_plan_ids):
            vehicle_id = vehicles[i % len(vehicles)]['id']
            job_queues[vehicle_id].append(job_map[job_id])

        completed_jobs = set()
        current_time = initial_states[0]['available_time']
        time_step = timedelta(seconds=60)  # Use a slightly larger time step for faster simulation
        active_edge_usage = {(u, v): 0 for u, v in graph.edges()}
        system_graph = graph
        system_forecast = ground_truth_weather[current_time : current_time + timedelta(hours=8)]

        # --- Event Tracking ---
        events.sort(key=lambda e: e['time'])
        next_event_idx = 0

        # --- Main Simulation Loop ---
        while len(completed_jobs) < len(jobs):
            # 1. EVENT TRIGGER CHECK
            if re_planning_enabled and next_event_idx < len(events) and current_time >= events[next_event_idx]['time']:
                event = events[next_event_idx]
                print(f"\n🔔 EVENT @ {current_time.strftime('%H:%M:%S')}: {event['description']} 🔔")
                next_event_idx += 1
                if 'forecast_update' in event: 
                    system_forecast = event['forecast_update']
                if 'graph_update' in event: 
                    system_graph = event['graph_update']

                print("   ▶ Pausing simulation for autonomous re-planning...")
                in_progress_jobs = {v['current_job']['id'] for v in vehicles if v.get('current_job')}
                queued_jobs = [job for queue in job_queues.values() for job in queue]
                jobs_to_replan = [job for job in queued_jobs if job['id'] not in in_progress_jobs]

                if jobs_to_replan:
                    for v in vehicles: 
                        v['available_time'] = current_time
                    _, new_plan_ids = optimizer_func(jobs_to_replan, vehicles, system_forecast, gnn_model, system_graph, weather_aware)
                    for q in job_queues.values(): 
                        q.clear()
                    for i, job_id in enumerate(new_plan_ids):
                        vehicle_id = vehicles[i % len(vehicles)]['id']
                        job_queues[vehicle_id].append(job_map[job_id])
                    print(f"   ✅ New plan with {len(new_plan_ids)} jobs adopted. Resuming simulation.")
                else:
                    print("   ...No queued jobs to replan. Resuming simulation.")
                print("-" * 30)

            # 2. VEHICLE STATE MACHINE LOGIC
            for v in vehicles:
                if v['status'] == 'moving_on_edge' and current_time >= v['edge_finish_time']:
                    if len(v['path']) > 1:
                        prev_edge = (v['path'][0], v['path'][1])
                        active_edge_usage[prev_edge] = max(0, active_edge_usage[prev_edge] - 1)
                        v['location'] = v['path'][1]
                        v['path'] = v['path'][1:]
                        v['status'] = 'en_route'
                        v['edge_finish_time'] = None

            for v in vehicles:
                if v['status'] in ['idle', 'en_route'] and len(v.get('path', [])) <= 1:
                    v['path'] = []
                    if v.get('current_job') and v['location'] == v['current_job']['destination']:
                        completed_jobs.add(v['current_job']['id'])
                        v['current_job'] = None
                    if not v.get('current_job') and job_queues[v['id']]:
                        v['current_job'] = job_queues[v['id']].pop(0)
                        v['path'] = self.safe_path_calculation(system_graph, v['location'], v['current_job']['source'])
                        if v['path'] is None:
                            v['status'] = 'blocked'
                            continue
                        v['status'] = 'en_route'
                    elif v.get('current_job') and v['location'] == v['current_job']['source']:
                        v['path'] = self.safe_path_calculation(system_graph, v['location'], v['current_job']['destination'])
                        if v['path'] is None:
                            v['status'] = 'blocked'
                            continue
                        v['status'] = 'en_route'
                    else:
                        v['status'] = 'idle'

            for v in vehicles:
                if v['status'] == 'en_route' and len(v['path']) > 1:
                    u, next_node = v['path'][0], v['path'][1]
                    edge = (u, next_node)
                    if not graph.has_edge(u, next_node):
                        v['status'] = 'blocked'
                        continue

                    edge_data = {**graph.edges[edge], 'start_node': u, 'end_node': next_node}
                    weather_now = SyntheticDataGenerator.get_weather_at_time(current_time, ground_truth_weather)
                    soil = 0
                    if not edge_data['paved'] and weather_now['rain_intensity'] > 1: 
                        soil = 2

                    # Use physics engine for travel time calculation
                    data_generator = SyntheticDataGenerator(graph, ground_truth_weather, {})
                    mean_time, _, _ = data_generator.calculate_ground_truth_travel(
                        edge_data, active_edge_usage[edge], weather_now, v['type'], current_time, soil)
                    
                    v['status'] = 'moving_on_edge'
                    v['edge_finish_time'] = current_time + timedelta(seconds=mean_time)
                    active_edge_usage[edge] += 1

            # 3. ADVANCE SIMULATION CLOCK
            current_time += time_step
            if current_time > initial_states[0]['available_time'] + timedelta(hours=24):
                print(f"🛑 SIMULATION TIMEOUT after 24 hours. {len(completed_jobs)}/{len(jobs)} jobs completed.")
                break

        # Calculate final makespan
        finish_times = [v['edge_finish_time'] for v in vehicles if v.get('edge_finish_time')]
        if finish_times:
            last_finish_time = max(finish_times + [current_time])
        else:
            last_finish_time = current_time
            
        makespan = (last_finish_time - initial_states[0]['available_time']).total_seconds() / 3600.0
        print(f"🏁 Simulation Finished. Makespan: {makespan:.2f} Hours")
        return makespan

# Initialize simulation manager
simulation_manager = AdaptiveSimulation(site_graph, weather_df)

In [ ]:
import random
import matplotlib.pyplot as plt
import copy
import numpy as np

class ExperimentRunner:
    """Runs comprehensive experiments comparing different optimization approaches"""
    
    def __init__(self, site_graph, weather_df, simulation_manager):
        self.site_graph = site_graph
        self.weather_df = weather_df
        self.simulation_manager = simulation_manager
        
        # Define vehicle fleet
        self.INITIAL_VEHICLE_STATES = [
            {'id': 0, 'type': 'truck', 'location': 'Main_Gate', 'available_time': SyntheticDataGenerator.SIMULATION_START_TIME},
            {'id': 1, 'type': 'truck', 'location': 'Contractor_Gate', 'available_time': SyntheticDataGenerator.SIMULATION_START_TIME},
            {'id': 2, 'type': 'forklift', 'location': 'Laydown_A', 'available_time': SyntheticDataGenerator.SIMULATION_START_TIME},
        ]
        
        # Define complex job scenarios
        self.COMPLEX_JOBS = [
            {'task': 'Deliver Rebar', 'sequence': [('Laydown_A', 'Building_1_N')]},
            {'task': 'Clear Debris from Pit', 'sequence': [('Foundation_Pit', 'Waste_Disposal')]},
            {'task': 'Move Scaffolding', 'sequence': [('Laydown_B', 'Building_2_W')]},
            {'task': 'Prefab Wall Delivery', 'sequence': [('Fabrication_Yard', 'Building_1_S')]},
            {'task': 'Multi-stop Debris Clearance', 'sequence': [('Building_1_S', 'Waste_Disposal'), ('Building_2_E', 'Waste_Disposal')]},
            {'task': 'Small Tool Delivery', 'sequence': [('Laydown_A', 'Foundation_Pit')]},
            {'task': 'Deliver piping', 'sequence': [('Laydown_A', 'Building_2_W')]},
            {'task': 'Move excess soil', 'sequence': [('Foundation_Pit', 'Waste_Disposal')]},
            {'task': 'Transport fabricated component', 'sequence': [('Fabrication_Yard', 'Building_2_E')]}
        ]
        
        self.JOBS_FOR_PLANNING = self._flatten_jobs()
        
    def _flatten_jobs(self):
        """Flatten complex multi-stop jobs into single-leg tasks"""
        jobs_for_planning = []
        job_counter = 0
        for job in self.COMPLEX_JOBS:
            for u, v in job['sequence']:
                jobs_for_planning.append({'id': job_counter, 'source': u, 'destination': v})
                job_counter += 1
        return jobs_for_planning
    
    def setup_disruption_scenario(self):
        """Set up realistic disruption scenario with weather and path closures"""
        INITIAL_FORECAST = self.weather_df[SyntheticDataGenerator.SIMULATION_START_TIME : 
                                         SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=8)]

        # The actual weather includes a surprise storm
        ground_truth_weather = self.weather_df.copy()
        storm_start = SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=2)
        storm_end = SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=4)
        ground_truth_weather.loc[storm_start:storm_end, 'rain'] = 8.0
        ground_truth_weather.loc[storm_start:storm_end, 'rain_intensity'] = 3
        print(f"Disruption 1: A surprise storm will occur between {storm_start.strftime('%H:%M')} and {storm_end.strftime('%H:%M')}.")

        # The new forecast, revealing the storm, becomes available at T+45min
        updated_forecast = ground_truth_weather[SyntheticDataGenerator.SIMULATION_START_TIME : 
                                              SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=8)]

        # A critical path on the main circulation loop is blocked by a crane at T+3h
        broken_down_graph = self.site_graph.copy()
        blocked_edge = ('Building_1_S', 'Main_Gate')
        if broken_down_graph.has_edge(*blocked_edge):
            broken_down_graph.remove_edge(*blocked_edge)
            print(f"Disruption 2: Path '{blocked_edge[0]}'->'{blocked_edge[1]}' will be blocked at { (SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=3)).strftime('%H:%M') }.")

        # Package disruptions into events
        simulation_events = [
            {
                'time': SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(minutes=45),
                'description': "Weather Forecast Update: Major Storm Predicted!",
                'forecast_update': updated_forecast
            },
            {
                'time': SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=3),
                'description': "Site Disruption: Main Gate Access Route Blocked!",
                'graph_update': broken_down_graph
            }
        ]
        
        return INITIAL_FORECAST, ground_truth_weather, updated_forecast, broken_down_graph, simulation_events
    
    def run_all_experiments(self, single_obj_optimizer, multi_obj_optimizer, static_optimizer):
        """Run comprehensive comparison of all optimization approaches"""
        print(f"✅ Complex scenario defined with {len(self.INITIAL_VEHICLE_STATES)} vehicles and {len(self.JOBS_FOR_PLANNING)} total tasks.")
        
        # Setup disruption scenario
        INITIAL_FORECAST, ground_truth_weather, updated_forecast, broken_down_graph, simulation_events = self.setup_disruption_scenario()
        
        results = {}
        
        # --- Model 1: Static Baseline ---
        static_plan, static_plan_ids = static_optimizer.run_static_optimizer(
            self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, self.site_graph)
        results['Static Baseline'] = self.simulation_manager.run_adaptive_simulation(
            static_plan_ids, self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, self.site_graph, 
            ground_truth_weather, optimizer_func=None, gnn_model=None, weather_aware=False, 
            events=simulation_events)

        # --- Model 2: Weather-Agnostic GNN (Static Plan) ---
        agnostic_plan, agnostic_plan_ids = single_obj_optimizer_agnostic.run_ga_optimizer(
            self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, INITIAL_FORECAST)
        
        # --- Model 3: Weather-Aware GNN (Static Plan) ---
        aware_plan_static, aware_plan_static_ids = single_obj_optimizer_aware.run_ga_optimizer(
            self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, INITIAL_FORECAST)
        
        # --- Model 4: Single-Objective Adaptive Digital Twin ---
        results['GA-GNN (Aware, Adaptive)'] = self.simulation_manager.run_adaptive_simulation(
            aware_plan_static_ids, self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, self.site_graph, 
            ground_truth_weather, optimizer_func=single_obj_optimizer_aware.run_ga_optimizer, 
            gnn_model=gnn_model_aware, weather_aware=True, events=simulation_events)

        # --- Model 5: Multi-Objective Adaptive Digital Twin ---
        print("\n--- Generating Initial Plan from Pareto Front ---")
        pareto_front = multi_obj_optimizer.run_nsga2_optimizer(
            self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, INITIAL_FORECAST, 
            gnn_model_aware, self.site_graph)

        if pareto_front:
            solutions = np.array([list(ind.fitness.values) for ind in pareto_front])
            fastest_idx = np.argmin(solutions[:, 0])
            fastest_plan_ids = list(pareto_front[fastest_idx])
            print(f"Selected 'Fastest' plan from Pareto front as the initial plan.")

            def replan_with_nsga2_fastest(jobs, states, weather, gnn, graph, weather_aware):
                """Wrapper for NSGA-II re-planning that selects fastest solution"""
                hof = multi_obj_optimizer.run_nsga2_optimizer(jobs, states, weather, gnn, graph)
                if not hof: 
                    return [], []
                new_solutions = np.array([list(ind.fitness.values) for ind in hof])
                new_fastest_idx = np.argmin(new_solutions[:, 0])
                best_plan_ids = list(hof[new_fastest_idx])
                dummy_plan = [[] for _ in states]
                for i, job_id in enumerate(best_plan_ids):
                    dummy_plan[i % len(states)].append(job_id)
                return dummy_plan, best_plan_ids

            results['Multi-Obj (Fastest, Adaptive)'] = self.simulation_manager.run_adaptive_simulation(
                fastest_plan_ids, self.JOBS_FOR_PLANNING, self.INITIAL_VEHICLE_STATES, self.site_graph, 
                ground_truth_weather, optimizer_func=replan_with_nsga2_fastest, 
                gnn_model=gnn_model_aware, weather_aware=True, events=simulation_events)
        else:
            print("WARNING: Pareto front is empty. Skipping Multi-Objective Champion model.")

        return results
    
    def visualize_results(self, results):
        """Visualize experiment results with comprehensive analysis"""
        sorted_results = sorted(results.items(), key=lambda item: item[1])
        print("\n\n" + "="*60)
        print("--- 📊 FINAL PERFORMANCE COMPARISON 📊 ---")
        print("="*60)
        for name, makespan in sorted_results:
            print(f"{name:<35}: {makespan:.2f} Hours")
        print("="*60)

        labels = [name for name, score in sorted_results]
        scores = [score for name, score in sorted_results]
        colors = ['grey', 'lightcoral', 'skyblue', 'darkorange', 'forestgreen']
        
        plt.figure(figsize=(14, 8))
        bars = plt.barh(labels, scores, color=colors[:len(labels)])
        plt.xlabel("Total Project Makespan (Hours)")
        plt.ylabel("Planning & Control Model")
        plt.title("Figure: Final Model Performance in Disrupted Scenario", fontsize=16)
        plt.gca().invert_yaxis()
        plt.tight_layout()

        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width + 0.05, bar.get_y() + bar.get_height()/2, f'{width:.2f} hrs', 
                    ha='left', va='center', fontsize=11, weight='bold')

        # Calculate improvement metrics
        try:
            baseline_score = results['GA-GNN (Aware, Static)']
            if 'Multi-Obj (Fastest, Adaptive)' in results:
                champion_score = results['Multi-Obj (Fastest, Adaptive)']
                improvement = ((baseline_score - champion_score) / baseline_score) * 100
                print(f"\n🏆 The adaptive multi-objective model showed a {improvement:.2f}% improvement over the best static plan.")
        except (KeyError, ZeroDivisionError):
            print("\nCould not calculate final improvement metric.")

        plt.xlim(right=max(scores) * 1.20)
        plt.show()

# Run comprehensive experiments
experiment_runner = ExperimentRunner(site_graph, weather_df, simulation_manager)
results = experiment_runner.run_all_experiments(single_obj_optimizer_aware, multi_obj_optimizer, static_optimizer)
experiment_runner.visualize_results(results)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

class ParetoAnalyzer:
    """Analyzes and visualizes Pareto front solutions"""
    
    def __init__(self, multi_obj_optimizer, jobs, states, weather_forecast):
        self.multi_obj_optimizer = multi_obj_optimizer
        self.jobs = jobs
        self.states = states
        self.weather_forecast = weather_forecast
        
    def analyze_pareto_front(self):
        """Run NSGA-II and analyze the resulting Pareto front"""
        print("--- Generating and Analyzing Pareto Front ---")
        pareto_front = self.multi_obj_optimizer.run_nsga2_optimizer(
            self.jobs, self.states, self.weather_forecast, gnn_model_aware, site_graph)

        if not pareto_front:
            print("Execution Warning: No solutions were found by the NSGA-II optimizer.")
            return None

        # Extract the fitness values (Time, Cost, Risk) from each non-dominated solution
        solutions = np.array([list(ind.fitness.values) for ind in pareto_front])

        # Separate the objectives into individual arrays for analysis and plotting
        times = solutions[:, 0]  # Makespan in Hours
        costs = solutions[:, 1]  # Total Fuel in Liters
        risks = solutions[:, 2]  # Sum of Variances (Risk Score)

        # Identify three key representative solutions from the front
        fastest_idx = np.argmin(times)
        cheapest_idx = np.argmin(costs)
        safest_idx = np.argmin(risks)

        best_time_solution = solutions[fastest_idx]
        best_cost_solution = solutions[cheapest_idx]
        best_risk_solution = solutions[safest_idx]

        print("\n" + "="*50)
        print("--- Analysis of Representative Optimal Solutions ---")
        print("="*50)
        print(f"Fastest Plan (Greedy):")
        print(f"  Makespan: {best_time_solution[0]:.2f} hrs | Fuel Cost: {best_time_solution[1]:.2f} L | Risk Score: {best_time_solution[2]:.2f}")
        print("\n")
        print(f"Cheapest Plan (Economical):")
        print(f"  Makespan: {best_cost_solution[0]:.2f} hrs | Fuel Cost: {best_cost_solution[1]:.2f} L | Risk Score: {best_cost_solution[2]:.2f}")
        print("\n")
        print(f"Safest Plan (Reliable/Low-Risk):")
        print(f"  Makespan: {best_risk_solution[0]:.2f} hrs | Fuel Cost: {best_risk_solution[1]:.2f} L | Risk Score: {best_risk_solution[2]:.2f}")
        print("="*50)

        return pareto_front, solutions, (best_time_solution, best_cost_solution, best_risk_solution)
    
    def visualize_pareto_3d(self, solutions, representative_solutions):
        """Create 3D visualization of Pareto front"""
        times, costs, risks = solutions[:, 0], solutions[:, 1], solutions[:, 2]
        best_time, best_cost, best_risk = representative_solutions

        fig = plt.figure(figsize=(14, 12))
        ax = fig.add_subplot(111, projection='3d')

        # Create the 3D scatter plot of all solutions on the Pareto front
        scatter = ax.scatter(times, costs, risks, c=times, cmap='viridis', s=60, alpha=0.8, edgecolors='k', linewidth=0.5)

        # Highlight the representative solutions with larger markers and labels
        ax.scatter(best_time[0], best_time[1], best_time[2], c='red', s=200, marker='*', label='Fastest')
        ax.text(best_time[0]*1.01, best_time[1]*1.01, best_time[2]*1.01, "Fastest", color='red', fontsize=12)

        ax.scatter(best_cost[0], best_cost[1], best_cost[2], c='blue', s=200, marker='P', label='Cheapest')
        ax.text(best_cost[0]*1.01, best_cost[1]*1.01, best_cost[2]*1.01, "Cheapest", color='blue', fontsize=12)

        ax.scatter(best_risk[0], best_risk[1], best_risk[2], c='green', s=200, marker='D', label='Safest')
        ax.text(best_risk[0]*1.01, best_risk[1]*1.01, best_risk[2]*1.01, "Safest", color='green', fontsize=12)

        # Set titles and labels for clarity
        ax.set_title('Pareto Front: Optimal Trade-offs (Time vs. Cost vs. Risk)', fontsize=18, pad=20)
        ax.set_xlabel('\nMakespan (Hours) - Lower is Better', fontsize=12)
        ax.set_ylabel('\nTotal Fuel (Liters) - Lower is Better', fontsize=12)
        ax.set_zlabel('\nLateness Risk Score - Lower is Better', fontsize=12)

        # Invert axes so that the "best" point (0,0,0) is visually in the corner
        ax.invert_xaxis()
        ax.invert_yaxis()
        ax.invert_zaxis()

        # Add a color bar to explain the color mapping
        cbar = fig.colorbar(scatter, shrink=0.6, aspect=20, pad=0.01)
        cbar.set_label('Makespan (Hours)', fontsize=12)

        ax.legend(fontsize=12)
        plt.show()
        
    def generate_green_vs_lean_table(self, representative_solutions):
        """Generate quantitative analysis of sustainability trade-offs"""
        best_time, best_cost, _ = representative_solutions
        
        # Define the CO2 conversion factor (kg of CO2 per liter of diesel)
        CO2_FACTOR = 2.68

        # Calculate metrics for the table
        lean_makespan = best_time[0]
        lean_fuel = best_time[1]
        lean_co2 = lean_fuel * CO2_FACTOR

        green_makespan = best_cost[0]
        green_fuel = best_cost[1]
        green_co2 = green_fuel * CO2_FACTOR

        # Calculate the percentage trade-offs
        makespan_increase_pct = ((green_makespan - lean_makespan) / lean_makespan) * 100
        co2_reduction_pct = ((lean_co2 - green_co2) / lean_co2) * 100

        # Create a Pandas DataFrame for clean formatting
        tradeoff_data = {
            'Metric': ['Project Makespan (Hours)', 'Total Fuel Consumed (Liters)', 'Total CO₂ Emissions (kg)'],
            'Lean Plan (Fastest)': [f"{lean_makespan:.2f}", f"{lean_fuel:.2f}", f"{lean_co2:.2f}"],
            'Green Plan (Most Fuel-Efficient)': [f"{green_makespan:.2f}", f"{green_fuel:.2f}", f"{green_co2:.2f}"],
            'Trade-Off': [f"+{makespan_increase_pct:.1f}% Time", "-", f"-{co2_reduction_pct:.1f}% Emissions"]
        }
        tradeoff_df = pd.DataFrame(tradeoff_data)

        print("\n\n" + "="*70)
        print("--- 🌍 Table: The Green vs. Lean Trade-Off in Construction Logistics 🌍 ---")
        print("="*70)
        print(tradeoff_df.to_markdown(index=False))
        print("="*70)
        print(f"\nKey Finding: Opting for the 'Green' plan reduces CO₂ emissions by {co2_reduction_pct:.1f}% at the cost of a {makespan_increase_pct:.1f}% increase in project makespan.")

# Run Pareto analysis
INITIAL_FORECAST = weather_df[SyntheticDataGenerator.SIMULATION_START_TIME : 
                             SyntheticDataGenerator.SIMULATION_START_TIME + timedelta(hours=8)]

# Create experiment runner to get jobs and states
experiment_runner = ExperimentRunner(site_graph, weather_df, simulation_manager)

pareto_analyzer = ParetoAnalyzer(multi_obj_optimizer, experiment_runner.JOBS_FOR_PLANNING, 
                               experiment_runner.INITIAL_VEHICLE_STATES, INITIAL_FORECAST)

pareto_results = pareto_analyzer.analyze_pareto_front()
if pareto_results:
    pareto_front, solutions, representative_solutions = pareto_results
    pareto_analyzer.visualize_pareto_3d(solutions, representative_solutions)
    pareto_analyzer.generate_green_vs_lean_table(representative_solutions)